In [364]:
import numpy as np
from collections import defaultdict
import math

In [365]:
with open('data/hw8_movieTitles.txt','r') as f:
    movies=f.read().split()
with open('data/hw8_studentPIDs.txt','r') as f:
    students=f.read().split()

In [366]:
ratings=[]
count=0
with open('data/hw8_ratings.txt','r') as f:
    temp=f.read().split('\n')
    for i in temp:
        i=i.replace('?','-1')
        temp1 = i.strip().split(' ')
        temp1=[int(j) for j in temp1]
        ratings.append(temp1)
lratings=ratings
ratings=np.matrix(ratings)

In [367]:
vis = defaultdict(list)
for i,j in enumerate(lratings):
    vis[i]=[]
    for k in range(len(j)):
        if j[k]==1 or j[k]==0:
            vis[i].append(k)

In [368]:
ls=len(students)
lm=len(movies)
moviepop=[]
for i in range(lm):
    temp=ratings[:,i].flatten().tolist()[0]
    moviepop.append(temp.count(1)*1.0/(temp.count(1)+temp.count(0)))
moviespopular=sorted(zip(moviepop,movies),key=lambda i:i[0])
for i in moviespopular:
    print '{0:45s} {1:.3f}'.format(i[1],i[0])    

Fifty_Shades_of_Grey                          0.331
The_Last_Airbender                            0.357
Magic_Mike                                    0.525
Prometheus                                    0.544
Bridemaids                                    0.556
World_War_Z                                   0.590
Man_of_Steel                                  0.592
Mad_Max:_Fury_Road                            0.633
Drive                                         0.636
Thor                                          0.665
Pitch_Perfect                                 0.667
The_Hunger_Games                              0.671
Fast_Five                                     0.678
The_Hateful_Eight                             0.683
Iron_Man_2                                    0.692
The_Perks_of_Being_a_Wallflower               0.695
American_Hustle                               0.716
The_Help                                      0.725
Avengers:_Age_of_Ultron                       0.730
21_Jump_Stre

In [369]:
priorZ=[]
with open('data/hw8_probZ_init.txt','r') as f:
    temp=f.read().strip().split('\n')
    for i in temp:
        priorZ.append(float(i)) 

In [370]:
rgz=[]
with open('data/hw8_probRgivenZ_init.txt','r') as f:
    temp=f.read().strip().split('\n')
    for i in temp:
        rgz.append([float(k) for k in i.split()])
#rgz=np.matrix(rgz)

In [371]:
pzt=defaultdict()
for i in range(len(students)):
    pzt[i]=defaultdict()
    for j in range(len(priorZ)):
        pzt[i][j]=priorZ[j]

rgzd=defaultdict()
for i in range(len(movies)):
    rgzd[i]=defaultdict()
    for j in range(len(priorZ)):
        rgzd[i][j]=rgz[i][j]

In [373]:
priorZ={0:0.25,1:0.25,2:0.25,3:0.25}

for i in rgzd:
    for k in rgzd[i]:
        tmp=0
        for p in range(len(movies)):
            if p in viz[i]:
                if ratings[i][p]==1:
                    tmp+=pzt[k][i]
            else:
                tmp+=pzt[k][i]*rgzd[p][k]
        rgzd[i][k]=temp/len(students)/priorZ[k]

In [374]:
import copy

def estep(priorZ,pzt,rgzd,vis):
    priorzt=defaultdict()
    for i in vis:
        temp1=0
        priorzt[i]=defaultdict()
        for j in priorZ:
            temp=priorZ[j]*1.0
            for k in vis[i]:
                if(ratings[i,k]==1):
                    temp*=rgzd[k][j]
                elif(ratings[i,k]==0):
                    temp*=(1-rgzd[k][j])
                else:
                    print "Fatal error"
                    return 0
            temp1+=temp
            priorzt[i][j]=temp

        for g in priorzt[i]:
            priorzt[i][g]/=temp1*1.0
        
    return priorZ,pzt,rgzd,vis,priorzt
    
    
def mstep(priorZ,pzt,rgzd,vis,priorzt):          
    
    divider=defaultdict()
    for j in priorZ:
        temp=0 
        for i in priorzt:
            temp+=priorzt[i][j]
        priorZ[j]=temp*1.0/len(students)
        divider[j]=temp
        
    old_rgzd = copy.deepcopy(rgzd)
    for i in rgzd:
        for p in rgzd[i]:
            rgzd[i][p]=0
            for t in vis:
                if(i in vis[t] and ratings[t,i]==1):
                    rgzd[i][p]+=priorzt[t][p]*1.0
                elif(i not in vis[t]):
                    rgzd[i][p]+=priorzt[t][p]*old_rgzd[i][p]*1.0
            rgzd[i][p]/=divider[p]*1.0

    return priorZ,pzt,rgzd,vis

def ll(priorZ,pzt,rgzd,vis):
    temp2=0
    for i in vis:
        temp1=0
        for j in priorZ:
            temp=priorZ[j]*1.0
            for k in vis[i]:
                if(ratings[i,k]==1):
                    temp*=rgzd[k][j]
                elif(ratings[i,k]==0):
                    temp*=(1-rgzd[k][j])
                else:
                    print "Fatal error"
                    return 0
            temp1+=temp
        temp2+=math.log(temp1)
    return temp2/len(students)

In [384]:
for ite in range(64):
    print ite,'\t',ll(priorZ,pzt,rgzd,vis)
    priorZ,pzt,rgzd,vis,priorzt=estep(priorZ,pzt,rgzd,vis)
    priorZ,pzt,rgzd,vis=mstep(priorZ,pzt,rgzd,vis,priorzt)

0 	-11.5401291391
1 	-11.5400122462
2 	-11.5399045735
3 	-11.5398059102
4 	-11.539715952
5 	-11.539634305
6 	-11.5395604976
7 	-11.5394939967
8 	-11.5394342262
9 	-11.5393805866
10 	-11.5393324754
11 	-11.5392893053
12 	-11.5392505186
13 	-11.5392155983
14 	-11.5391840745
15 	-11.5391555275
16 	-11.5391295874
17 	-11.5391059325
18 	-11.5390842853
19 	-11.5390644086
20 	-11.5390461004
21 	-11.5390291896
22 	-11.5390135312
23 	-11.5389990026
24 	-11.5389854991
25 	-11.5389729316
26 	-11.538961223
27 	-11.5389503064
28 	-11.5389401229
29 	-11.5389306199
30 	-11.5389217501
31 	-11.5389134705
32 	-11.5389057413
33 	-11.5388985258
34 	-11.5388917895
35 	-11.5388855
36 	-11.5388796268
37 	-11.5388741411
38 	-11.5388690155
39 	-11.5388642243
40 	-11.538859743
41 	-11.5388555487
42 	-11.5388516197
43 	-11.5388479354
44 	-11.5388444768
45 	-11.5388412259
46 	-11.5388381659
47 	-11.538835281
48 	-11.5388325567
49 	-11.5388299793
50 	-11.5388275362
51 	-11.5388252156
52 	-11.5388230065
53 	-11.538

In [405]:
def reco(priorZ,pzt,rgzd,vis,priorzt):
    temp1=0
    i=42
    ptemp=defaultdict()
    for j in priorZ:
        temp=priorZ[j]*1.0
        for k in vis[i]:
            if(ratings[i,k]==1):
                temp*=rgzd[k][j]
            elif(ratings[i,k]==0):
                temp*=(1-rgzd[k][j])
            else:
                print "Fatal error"
                return 0
        temp1+=temp
        ptemp[j]=temp

    for g in ptemp:
        ptemp[g]/=temp1*1.0
        
    moviesreco=defaultdict()
    for li in range(50):
        if(li not in vis[i]):
            temp=0
            for k in priorZ:
                temp+=ptemp[k]*rgzd[li][k]
            moviesreco[li]=temp
    return moviesreco

In [406]:
moviesr = reco(priorZ,pzt,rgzd,vis,priorzt)

In [419]:
for i in sorted(moviesr.items(),key=lambda k : -k[1]):
    print "{0:50s} with probability : {1:10f}".format(movies[i[0]],i[1])

American_Hustle                                    with probability :   0.887529
Les_Miserables                                     with probability :   0.852486
Drive                                              with probability :   0.852315
Midnight_in_Paris                                  with probability :   0.808777
The_Girls_with_the_Dragon_Tattoo                   with probability :   0.781265
Magic_Mike                                         with probability :   0.615558
The_Great_Gatsby                                   with probability :   0.594119
Frozen                                             with probability :   0.560564
Thor                                               with probability :   0.470917
Pitch_Perfect                                      with probability :   0.421858
Toy_Story_3                                        with probability :   0.321624
Room                                               with probability :   0.285091
Bridemaids                  